In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, SimpleRNN, Masking, TimeDistributed, LSTM
import numpy as np
import datetime as dt

Using TensorFlow backend.


In [4]:
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
#tf.test.gpu_device_name()

In [2]:
DATA_PATH = "../TwitchHighlightCrawler/vod/" # const
DUMMY_VALUE = -1.0

In [3]:
X = np.load('training_data-k8uc6gtfz5tul.npy')
Y = np.load('training_label-k8uc6gtfz5tul.npy')

In [4]:
def stringToDateTime(str): # only parse twitch info format
    str = (str[:19] + 'Z') if len(str) > 20 else str
    return dt.datetime.strptime(str, '%Y-%m-%dT%H:%M:%SZ')

In [5]:
model = Sequential()
model.add(Masking(mask_value=DUMMY_VALUE, input_shape=(None, 2)))
#model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(1, return_sequences=True, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy')
#model.summary()
model.fit(X, Y, epochs=10, batch_size=8)
model.save('mmm.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, None, 2)           0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 1)           4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


W0813 01:04:24.592210 11348 deprecation_wrapper.py:119] From D:\Users\User\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
156/156 [==============================] - 811s 5s/step - loss: 1.1295
Epoch 2/10
156/156 [==============================] - 784s 5s/step - loss: 1.1192
Epoch 3/10
156/156 [==============================] - 782s 5s/step - loss: 1.0820
Epoch 4/10
156/156 [==============================] - 790s 5s/step - loss: 1.0549
Epoch 5/10
156/156 [==============================] - 793s 5s/step - loss: 1.0359
Epoch 6/10
156/156 [==============================] - 777s 5s/step - loss: 1.0469
Epoch 7/10
156/156 [==============================] - 811s 5s/step - loss: 1.0367
Epoch 8/10
156/156 [==============================] - 853s 5s/step - loss: 0.9932
Epoch 9/10
156/156 [==============================] - 838s 5s/step - loss: 0.9745
Epoch 10/10
156/156 [==============================] - 813s 5s/step - loss: 0.9862


In [4]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=10))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 生成虚拟数据
import numpy as np
'''data = np.random.random((1000, 10))
labels = np.random.randint(2, size=(1000, 1))'''
data = []
labels = []
import random
for i in range(1000):
    data.append(np.random.random(10))
    labels.append(random.randint(0,1))

# 训练模型，以 32 个样本为一个 batch 进行迭代
model.fit(data, labels, epochs=10, batch_size=32)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 1000 arrays: [array([[0.51024308],
       [0.3404787 ],
       [0.07502523],
       [0.26591401],
       [0.2502586 ],
       [0.28769308],
       [0.43627578],
       [0.19952613],
       [0.87965262],
       [0....

In [70]:
'''data = np.array([])
data = np.append(data, np.random.random((10, 2)), axis=0)
#data = np.append(data, np.random.random((10, 2)), axis=0)
#np.concatenate((data, np.random.random((1,10, 2))),axis=0 )
np.random.random((10, 2))
data'''
import random
d = []
features = []
for i in range(10):
    feature = []
    for j in range(2):
        feature.append(random.random())
    features.append(feature)
    
d.append(features)
d.append(features)
np.array(d)

array([[[0.95156836, 0.5257626 ],
        [0.00156089, 0.85387701],
        [0.78281018, 0.73223741],
        [0.37595671, 0.49269121],
        [0.92361927, 0.67693445],
        [0.9214292 , 0.72112536],
        [0.66214929, 0.10112643],
        [0.90462236, 0.31870872],
        [0.15810725, 0.73876466],
        [0.53127291, 0.89857455]],

       [[0.95156836, 0.5257626 ],
        [0.00156089, 0.85387701],
        [0.78281018, 0.73223741],
        [0.37595671, 0.49269121],
        [0.92361927, 0.67693445],
        [0.9214292 , 0.72112536],
        [0.66214929, 0.10112643],
        [0.90462236, 0.31870872],
        [0.15810725, 0.73876466],
        [0.53127291, 0.89857455]]])

In [3]:
# 生成虚拟数据
import random

data = []
labels = []
max_seq_len = 0

N = 10 # number of data

dummy = -1
for i in range(N):
    length = random.randint(5,25)
    if length > max_seq_len:
        max_seq_len = length
    #length = 10
    data.append( np.random.random((length, 2))  )
    labels.append( np.random.randint(2, size=(length, 1)) )
#np.set_printoptions(threshold=False)

Xpad = np.full((N, max_seq_len, 2), fill_value=dummy, dtype='float64')
Ypad = np.full((N, max_seq_len, 1), fill_value=dummy, dtype='float64')
for s, x in enumerate(data):
    video_length = len(x)
    Xpad[s, 0:video_length, :] = x
for s, y in enumerate(labels):
    video_length = len(y)
    Ypad[s, 0:video_length, :] = y

#data = np.array(data)
#labels = np.array(labels)
#print(data.shape)
#print(labels.shape)



#input = Input(shape=(None, 2))

model = Sequential()
model.add(Masking(mask_value=dummy, input_shape=(None, 2)))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(1, return_sequences=True, activation='sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
#model.add(TimeDistributed(Dense(2, activation='softmax')))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


'''data = []
with np.printoptions(threshold=np.inf):
    print(Xpad)
    print(data)'''
# 训练模型，以 32 个样本为一个 batch 进行迭代
#model.fit(data, labels, epochs=10, batch_size=8)
model.fit(Xpad, Ypad, epochs=10, batch_size=8)

W0821 19:51:54.256520  4772 deprecation_wrapper.py:119] From D:\Users\User\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0821 19:51:54.494101  4772 deprecation_wrapper.py:119] From D:\Users\User\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 19:51:54.571093  4772 deprecation_wrapper.py:119] From D:\Users\User\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0821 19:51:54.920798  4772 deprecation.py:323] From D:\Users\User\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend\tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorfl

Epoch 1/10
10/10 [==============================] - 10s 989ms/step - loss: 0.7155 - acc: 0.5360
Epoch 2/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6940 - acc: 0.5642
Epoch 3/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6936 - acc: 0.5517
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6891 - acc: 0.5585
Epoch 5/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6891 - acc: 0.5601
Epoch 6/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6850 - acc: 0.5660
Epoch 7/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6834 - acc: 0.5632
Epoch 8/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6814 - acc: 0.5998
Epoch 9/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6785 - acc: 0.5844
Epoch 10/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6771 - acc: 0.5918


In [8]:
test = []

'''import random
features = []
for i in range(10):
    feature = []
    for j in range(2):
        feature.append(random.random())
    features.append(feature)
test.append(features)
test1 = np.array(test)

test = []
features = []
for i in range(6):
    feature = []
    for j in range(2):
        feature.append(random.random())
    features.append(feature)
test.append(features)
test2 = np.array(test)

test = np.array(test)
with np.printoptions(threshold=np.inf):
    for i in range(2):
        print(model.predict(test, batch_size=8))'''

'''test = np.array([ [[[2,5],[3,6],[4,8]]], [[[2,5],[3,6]]] ]) # 4維 沒用
with np.printoptions(threshold=np.inf):
    print(model.predict(test[0], batch_size=8))'''

'''a = np.array([[[2,5],[3,6],[4,8]]])
b = np.array([[[2,5],[3,6]]])
c = []
c.append(a)
c.append(b)
test = np.array(c) # 無解 這行出錯
with np.printoptions(threshold=np.inf):
    print(model.predict(test[0], batch_size=8))'''

'''test.append(np.random.random((10, 2)))
test.append(np.random.random((6, 2)))
#test = np.array(test)
Tpad = np.full((2, 10, 2), fill_value=dummy, dtype='Float64')
for s, t in enumerate(test):
    video_length = len(t)
    Tpad[s, 0:video_length, :] = t

with np.printoptions(threshold=np.inf):
    print(model.predict(Tpad, batch_size=8))'''

[array([[[2, 5],
        [3, 6],
        [4, 8]]]), array([[[2, 5],
        [3, 6]]])]


ValueError: could not broadcast input array from shape (3,2) into shape (1)

In [7]:
def local_score(clipDataList, predict, numberOfPredictedFragment):
    localPrecision = 0
    localRecall = 0
    
    videoLength = len(predict)
    
    for clip in clipDataList:
        recall = 0
        overlapTime = 0 # for local recall computing

        endOfClip = clip['offset'] + clip['duration'] # not real video end timeline point

        for i in range(clip['offset'] - (MAX_CLIP_LENGTH-1), clip['offset'] - MIN_CLIP_LENGTH + 1): # (offset - 59) <= prediction <= (offset - 5) # eg. offset = 85, duration = 10, 26~80
            if predict[i] and (i + predict[i]['duration']) > clip['offset']: # Simplify from (i + (predict[i]['duration']-1)) >= clip['offset']
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfPrediction - clip['offset'] # compute overlap seconds

                if endOfPrediction > endOfClip:
                    overlap -= (endOfPrediction - endOfClip) # remove the part over than the end of ground true

                # precision
                predict[i]['precision'].append( overlap/predict[i]['duration'] )
                # local recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                overlapTime += 1

        for i in range(clip['offset'] - MIN_CLIP_LENGTH + 1, clip['offset']): # (offset - 4(less than MIN_CLIP_LENGTH)) <= prediction < offset # 81~84
            if predict[i]: # no need to check if it overlap the ground truth period, it must be in the period
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfPrediction - clip['offset']; # compute overlap seconds

                if endOfPrediction > endOfClip: # predicted fragment is larger than clip
                    overlap -= (endOfPrediction - endOfClip); # remove the part over than the end of ground true

                # precision
                predict[i]['precision'].append( overlap/predict[i]['duration'] )
                # local recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                overlapTime += 1

        for i in range(clip['offset'], endOfClip): # offset <= prediction < (offset + duration) # 85~94
            if predict[i]: # no need to check if it overlap the ground truth period, it must be in the period
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfClip - i; # compute overlap seconds

                if endOfPrediction < endOfClip: # predicted fragment is smaller than clip
                    overlap -= endOfClip - endOfPrediction

                # precision
                predict[i]['precision'].append( overlap/predict[i]['duration'] )
                # local recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                overlapTime += 1

        if overlapTime:
            recall /= overlapTime
            localRecall += recall
    
    for i in range(videoLength):
        if predict[i]: # check have predicted fragment
            if len(predict[i]['precision']): # the fragment has overlap
                precision = 0 # for local precision computing

                for singlePrecisionValue in predict[i]['precision']:
                    # local precision
                    precision += singlePrecisionValue
                precision /= len(predict[i]['precision'])

                localPrecision += precision

    localPrecision /= numberOfPredictedFragment
    localRecall /= len(clipDataList)
    F1 = (2 * localPrecision * localRecall) / (localPrecision + localRecall)
    print(str(localPrecision) + ' ' + str(localRecall) + ' ' + str(F1))
    
    return {
        'precision': precision,
        'recall': recall,
        'F1': F1
    }
    
def macro_score(clipDataList, predict):
    macroPrecision = 0
    macroRecall = 0
    macroF1 = 0
    
    videoLength = len(predict)
    
    for clip in clipDataList:
        precision = 0
        recall = 0
        F1 = 0
        
        overlapTime = 0

        endOfClip = clip['offset'] + clip['duration'] # not real video end timeline point

        for i in range(clip['offset'] - (MAX_CLIP_LENGTH-1), clip['offset'] - MIN_CLIP_LENGTH + 1): # (offset - 59) <= prediction <= (offset - 5) # eg. offset = 85, duration = 10, 26~80
            if predict[i] and (i + predict[i]['duration']) > clip['offset']: # Simplify from (i + (predict[i]['duration']-1)) >= clip['offset']
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfPrediction - clip['offset'] # compute overlap seconds

                if endOfPrediction > endOfClip:
                    overlap -= (endOfPrediction - endOfClip) # remove the part over than the end of ground true

                # precision
                singlePrecisionVlaue = overlap / predict[i]['duration']
                precision += singlePrecisionVlaue
                # recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                # F1
                singleF1Value = (2 * singlePrecisionVlaue * singleRecallValue) / (singlePrecisionVlaue + singleRecallValue)
                F1 += singleF1Value
                
                overlapTime += 1

        for i in range(clip['offset'] - MIN_CLIP_LENGTH + 1, clip['offset']): # (offset - 4(less than MIN_CLIP_LENGTH)) <= prediction < offset # 81~84
            if predict[i]: # no need to check if it overlap the ground truth period, it must be in the period
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfPrediction - clip['offset']; # compute overlap seconds

                if endOfPrediction > endOfClip: # predicted fragment is larger than clip
                    overlap -= (endOfPrediction - endOfClip); # remove the part over than the end of ground true

                # precision
                singlePrecisionVlaue = overlap / predict[i]['duration']
                precision += singlePrecisionVlaue
                # recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                # F1
                singleF1Value = (2 * singlePrecisionVlaue * singleRecallValue) / (singlePrecisionVlaue + singleRecallValue)
                F1 += singleF1Value
                
                overlapTime += 1

        for i in range(clip['offset'], endOfClip): # offset <= prediction < (offset + duration) # 85~94
            if predict[i]: # no need to check if it overlap the ground truth period, it must be in the period
                endOfPrediction = i + predict[i]['duration']
                overlap = endOfClip - i; # compute overlap seconds

                if endOfPrediction < endOfClip: # predicted fragment is smaller than clip
                    overlap -= endOfClip - endOfPrediction

                # precision
                singlePrecisionVlaue = overlap / predict[i]['duration']
                precision += singlePrecisionVlaue
                # recall
                singleRecallValue = overlap / clip['duration']
                recall += singleRecallValue
                # F1
                singleF1Value = (2 * singlePrecisionVlaue * singleRecallValue) / (singlePrecisionVlaue + singleRecallValue)
                F1 += singleF1Value
                
                overlapTime += 1

        if overlapTime:
            precision /= overlapTime
            recall /= overlapTime
            F1 /= overlapTime
            
            macroPrecision += precision
            macroRecall += recall
            macroF1 += F1
    
    numberOfAnswer = len(clipDataList)
    
    macroPrecision /= numberOfAnswer
    macroRecall /= numberOfAnswer
    macroF1 /= numberOfAnswer  
    print(str(macroPrecision) + ' ' + str(macroRecall) + ' ' + str(macroF1))
    
def K_simple_global_score(label, prediction): # use to evaluation training data
    THRESHOLD = 0.5
    
    overlap = 0
    totalClipSeconds = 0
    totalPredictionSeconds = 0
    
    length = len(label)
    for i in range(length):
        if label[i][0] == -1:
            break
            
        if prediction[i][0] > THRESHOLD:
            totalPredictionSeconds += 1
            if label[i][0]:
                totalClipSeconds += 1
                overlap += 1
        elif label[i][0]:
            totalClipSeconds += 1
            
    precision = (overlap / totalPredictionSeconds) if totalPredictionSeconds > 0 else 0
    recall = (overlap / totalClipSeconds) if totalClipSeconds > 0 else 0
    F1 = ((2 * precision * recall) / (precision + recall)) if (precision + recall) > 0 else 0
            
    return {
        'precision': precision,
        'recall': recall,
        'F1': F1
    }

def global_score(groundTruth, predict):
    videoLength = len(groundTruth)
    
    # normal score
    overlap = 0
    totalClipSeconds = 0 # WARNING!! we can count this value when computing local score
    totalPredictionSeconds = 0
    
    # weighted score
    weightedOverlap = 0
    totalWeightedClipSeconds = 0 # WARNING!! we can count this value when computing local score
    totalWeightedPredictionSeconds = 0
    
    for i in range(videoLength):
        if groundTruth[i]:
            totalClipSeconds += 1
            totalWeightedClipSeconds += groundTruth[i]
            
            if predict[i]:
                totalPredictionSeconds += 1
                totalWeightedPredictionSeconds += groundTruth[i]
                
                overlap += 1
                weightedOverlap += groundTruth[i]
        elif predict[i]:
            totalPredictionSeconds += 1
            totalWeightedPredictionSeconds += 1 # no clip on this second, so weight is 1
                
    precision = overlap / totalPredictionSeconds if totalPredictionSeconds > 0 else 0
    recall = overlap / totalClipSeconds if totalClipSeconds > 0 else 0
    F1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    weightedPrecision = weightedOverlap / totalWeightedPredictionSeconds if totalWeightedPredictionSeconds > 0 else 0
    weightedRecall = weightedOverlap / totalWeightedClipSeconds if totalWeightedClipSeconds > 0 else 0
    weightedF1 = (2 * weightedPrecision * weightedRecall) / (weightedPrecision + weightedRecall) if (weightedPrecision + weightedRecall) > 0 else 0
    
    #print(str(precision) + ' ' + str(recall) + ' ' + str(F1))
    #print(str(weightedPrecision) + ' ' + str(weightedRecall) + ' ' + str(weightedF1))
    
    return {
        'normal': {
            'precision': precision,
            'recall': recall,
            'F1': F1
        },
        'weighted': {
            'precision': weightedPrecision,
            'recall': weightedRecall,
            'F1': weightedF1
        }
    }

def K_global_score(groundTruth, prediction):
    THRESHOLD = 0.5
    videoLength = len(groundTruth)
    
    # normal score
    overlap = 0
    totalClipSeconds = 0
    totalPredictionSeconds = 0
    
    # weighted score
    weightedOverlap = 0
    totalWeightedClipSeconds = 0
    totalWeightedPredictionSeconds = 0
    
    for i in range(videoLength):
        if groundTruth[i]:
            totalClipSeconds += 1
            totalWeightedClipSeconds += groundTruth[i]
            
            if prediction[i][0] > THRESHOLD:
                totalPredictionSeconds += 1
                totalWeightedPredictionSeconds += groundTruth[i]
                
                overlap += 1
                weightedOverlap += groundTruth[i]
        elif prediction[i][0] > THRESHOLD:
            totalPredictionSeconds += 1
            totalWeightedPredictionSeconds += 1 # no clip on this second, so weight is 1
                
    precision = overlap / totalPredictionSeconds if totalPredictionSeconds > 0 else 0
    recall = overlap / totalClipSeconds if totalClipSeconds > 0 else 0
    F1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    weightedPrecision = weightedOverlap / totalWeightedPredictionSeconds if totalWeightedPredictionSeconds > 0 else 0
    weightedRecall = weightedOverlap / totalWeightedClipSeconds if totalWeightedClipSeconds > 0 else 0
    weightedF1 = (2 * weightedPrecision * weightedRecall) / (weightedPrecision + weightedRecall) if (weightedPrecision + weightedRecall) > 0 else 0
    
    #print(str(precision) + ' ' + str(recall) + ' ' + str(F1))
    #print(str(weightedPrecision) + ' ' + str(weightedRecall) + ' ' + str(weightedF1))
    
    return {
        'normal': {
            'precision': precision,
            'recall': recall,
            'F1': F1
        },
        'weighted': {
            'precision': weightedPrecision,
            'recall': weightedRecall,
            'F1': weightedF1
        }
    }

def evaluation(channel, video, predict): # WARNING!! it hasn't deal that answer is out of the video yet
    global_answer = np.zeros(len(predict['macro']), dtype=int) # WARNING!! use dependent data len(predict['macro'])
    
    clipList = os.listdir(DATA_PATH + channel + '/'+ video +'/clip')
    clipDataList = [] # ground true for loacl & macro
    for clip in clipList:
        with open(DATA_PATH + channel + '/'+ video +'/clip/' + clip, "r", encoding="utf-8") as file:
            data = file.read()

        data = json.loads(data)
        
        # generate ground true for local & macro evaluation metrics
        clipDataList.append({
            'offset': data['vod']['offset'],
            'duration': math.ceil( data['duration'] ) # length of ground true clip
        })
        
        # generate ground true for global evaluation metrics
        for i in range(clipDataList[-1]['offset'], clipDataList[-1]['offset'] + clipDataList[-1]['duration']):
            global_answer[i] += 1
    
    print(predict['count'])
    local_score(clipDataList, predict['macro'], predict['count'])
    #macro_score(clipDataList, predict['macro'])
    global_score(global_answer, predict['global'])
    
def evaluationGlobalOnly(channel, video, prediction, clip_deadline = dt.datetime(9999,12,31,23,59,59), filter_low_views = False):
    VIDEO_LENGTH = len(prediction) # WARNING!! use dependent data len(prediction)
    VIEWS_THRESHOLD = 3
    
    global_answer = np.zeros(VIDEO_LENGTH, dtype=int)
    
    CLIP_PATH = DATA_PATH + channel + '/'+ video +'/clip/'
    clipList = os.listdir(CLIP_PATH)
    for clip in clipList:
        with open(CLIP_PATH + clip, "r", encoding="utf-8") as file:
            data = file.read()
        clipInfo = json.loads(data)
        
        if filter_low_views and clipInfo['views'] < VIEWS_THRESHOLD:
            continue # filter out a clip with low views
        if stringToDateTime(clipInfo['created_at']) >= clip_deadline:
            continue # filter out a clip created outside the range
        
        offset = clipInfo['vod']['offset']
        duration = math.ceil( clipInfo['duration'] ) # length of ground true clip

        if offset >= VIDEO_LENGTH: # check a clip is locate in the video range
            continue

        # generate ground true for global evaluation metrics
        for i in range(offset, offset + duration):
            if i >= VIDEO_LENGTH:
                break
            global_answer[i] += 1
    
    #print(channel + ' ' + video)
    return K_global_score(global_answer, prediction)

In [ ]:
precision = 0
recall = 0
F1 = 0

weightedPrecision = 0
weightedRecall = 0
weightedF1 = 0

predictions = 
for mark, prediction in zip(TESTING_MARK, predictions):
    with open(DATA_PATH + mark[0] + '/' + mark[1] + '/info.json', "r", encoding="utf-8") as file: # mark[0]: channel, mark[1]: video
        data = file.read()
    videoInfo = json.loads(data)
    
    clipDeadline = stringToDateTime(videoInfo['created_at']) + dt.timedelta(seconds=videoInfo['length']) + dt.timedelta(days=CLIP_GRACE_PERIOD)
    
    scoreData = evaluationGlobalOnly(mark[0], mark[1], prediction[:videoInfo['length']], clipDeadline, True)
    precision += scoreData['normal']['precision']
    recall += scoreData['normal']['recall']
    F1 += scoreData['normal']['F1']
    weightedPrecision += scoreData['weighted']['precision']
    weightedRecall += scoreData['weighted']['recall']
    weightedF1 += scoreData['weighted']['F1']
    

precision /= len(predictions)
recall /= len(predictions)
F1 /= len(predictions)
weightedPrecision /= len(predictions)
weightedRecall /= len(predictions)
weightedF1 /= len(predictions)
print(str(precision) + ' ' + str(recall) + ' ' + str(F1))
print(str(weightedPrecision) + ' ' + str(weightedRecall) + ' ' + str(weightedF1))